# Feature Pipeline using Synthetic Data

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/featurestoreorg/serverless-ml-course/blob/main/src/02-module/2_cc_feature_pipeline.ipynb)

**Note**: you may get an error when installing hopsworks on Colab, and it is safe to ignore it.

## 🗒️ This notebook is divided in 2 sections:
1. Reading the synthetic credit card data and feature engineeing,
2. Write the Pandas DataFrames to the feature groups in the feature store.


In [2]:
#!pip install -U hopsworks --quiet
!pip install -U faker --quiet

In [1]:
import pandas as pd
import datetime
import hopsworks
from sml import synthetic_data
import random
pd.options.mode.chained_assignment = None

/home/codespace/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
start_time = (datetime.datetime.now() - datetime.timedelta(hours=24)).strftime("%Y-%m-%d %H:%M:%S")
print(start_time)

2024-03-21 12:48:22


In [3]:
#end_time = (datetime.datetime.now() - datetime.timedelta(hours=24)).strftime("%Y-%m-%d %H:%M:%S")
end_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(end_time)

2024-03-22 12:48:26


In [4]:
synthetic_data.FRAUD_RATIO = random.uniform(0.001, 0.005)
synthetic_data.TOTAL_UNIQUE_USERS = 1000
synthetic_data.TOTAL_UNIQUE_TRANSACTIONS = 54000
synthetic_data.CASH_WITHRAWAL_CARDS_TOTAL = 2000
synthetic_data.TOTAL_UNIQUE_CASH_WITHDRAWALS = 200
synthetic_data.START_DATE=start_time
synthetic_data.END_DATE=end_time

credit_cards = synthetic_data.generate_list_credit_card_numbers()
credit_cards_df = synthetic_data.create_credit_cards_as_df(credit_cards)
profiles_df = synthetic_data.create_profiles_as_df(credit_cards)
trans_df = synthetic_data.create_transactions_as_df(credit_cards)

## <span style="color:#ff5f27;"> 🛠️ Feature Engineering </span>

Fraudulent transactions can differ from regular ones in many different ways. Typical red flags would for instance be a large transaction volume/frequency in the span of a few hours. It could also be the case that elderly people in particular are targeted by fraudsters. To facilitate model learning you will create additional features based on these patterns. In particular, you will create two types of features:
1. **Features that aggregate data from different data sources**. This could for instance be the age of a customer at the time of a transaction, which combines the `birthdate` feature from `profiles.csv` with the `datetime` feature from `transactions.csv`.
2. **Features that aggregate data from multiple time steps**. An example of this could be the transaction frequency of a credit card in the span of a few hours, which is computed using a window function.

Let's start with the first category.

In [5]:
fraud_labels = trans_df.copy()[["tid", "cc_num", "datetime", "fraud_label"]]
fraud_labels

,tid,cc_num,datetime,fraud_label
0,7f9371d851d97e8a5d959edacd2308c9,4422695741483506,2024-03-21 12:48:23,0
1,b3718c2c6b1442b5bbc3a9d781e5c9a6,4222689788431660,2024-03-21 12:48:27,0
2,4e33f6d090b0efb76e0b0c8d83820eb7,4286362653607516,2024-03-21 12:48:27,0
3,25415da163fb7b983a9c48d3feee757e,4570139020102669,2024-03-21 12:48:28,0
4,61a490681bf64693f8dc06c658b08168,4201609549484043,2024-03-21 12:48:32,0
...,...,...,...,...
60081,6b6419722033d2de0fb8f2c5329e4010,4451523647410551,2024-04-09 12:28:03,0
60082,520031a38e54edeed1129c834cada631,4451523647410551,2024-04-12 14:28:03,0
60083,6004d15f6243b15a19724d9a2fffa74b,4451523647410551,2024-04-15 16:28:03,0
60084,b4e21ec2569c93df43184e86b661cc35,4451523647410551,2024-04-18 18:28:03,0


In [8]:
from sml import cc_features

fraud_labels.datetime = fraud_labels.datetime.map(lambda x: cc_features.date_to_timestamp(x))
fraud_labels

,tid,cc_num,datetime,fraud_label
0,7f9371d851d97e8a5d959edacd2308c9,4422695741483506,1711025303000,0
1,b3718c2c6b1442b5bbc3a9d781e5c9a6,4222689788431660,1711025307000,0
2,4e33f6d090b0efb76e0b0c8d83820eb7,4286362653607516,1711025307000,0
3,25415da163fb7b983a9c48d3feee757e,4570139020102669,1711025308000,0
4,61a490681bf64693f8dc06c658b08168,4201609549484043,1711025312000,0
...,...,...,...,...
60081,6b6419722033d2de0fb8f2c5329e4010,4451523647410551,1712665683000,0
60082,520031a38e54edeed1129c834cada631,4451523647410551,1712932083000,0
60083,6004d15f6243b15a19724d9a2fffa74b,4451523647410551,1713198483000,0
60084,b4e21ec2569c93df43184e86b661cc35,4451523647410551,1713464883000,0


In [9]:
trans_df

,tid,datetime,cc_num,category,amount,latitude,longitude,city,country,fraud_label
0,9e7f110e7a20645f78e9ecb14334f5e3,2024-03-20 13:14:48,4266382731398114,Electronics,456.97,36.025060,-86.779170,Brentwood Estates,US,0
1,08daf9d62a167c3df1f12ff502276f8c,2024-03-20 13:14:49,4322428180207298,Grocery,71.93,33.410120,-91.061770,Greenville,US,0
2,061c39bcdf76e02b5c3cdcee4b976b26,2024-03-20 13:14:50,4709001521516926,Restaurant/Cafeteria,10.87,33.036990,-117.291980,Encinitas,US,0
3,aac14fa593c1bc877a5e26149d2b0d1a,2024-03-20 13:14:52,4160440205525104,Grocery,79.64,29.845760,-90.106740,Estelle,US,0
4,fa59fafdab41aef697197b52d6c29f3e,2024-03-20 13:14:55,4610739772483860,Holliday/Travel,43.42,40.605380,-73.755130,Far Rockaway,US,0
...,...,...,...,...,...,...,...,...,...,...
60081,1b402606d739bf3b2ff834845713d998,2024-04-08 01:26:56,4267422077883458,Cash Withdrawal,62.36,34.239010,-119.044274,Camarillo,US,0
60082,f7660e16c8f381f6b0db24ff6f6918de,2024-04-11 03:26:56,4267422077883458,Cash Withdrawal,79.94,34.248261,-119.041913,Camarillo,US,0
60083,3b610ed4e050949da57871cb7c9a904e,2024-04-14 05:26:56,4267422077883458,Cash Withdrawal,2.69,34.243697,-119.033705,Camarillo,US,0
60084,2d149243790e7a5e8c99e87859e779c2,2024-04-17 07:26:56,4267422077883458,Cash Withdrawal,16.80,34.234965,-119.040798,Camarillo,US,0


In [6]:
trans_df.drop(['fraud_label'], inplace = True, axis=1)

In [9]:
trans_df = cc_features.card_owner_age(trans_df, profiles_df)
trans_df = cc_features.expiry_days(trans_df, credit_cards_df)
trans_df = cc_features.activity_level(trans_df, 1)

In [10]:
window_len = 4
window_aggs_df = cc_features.aggregate_activity_by_hour(trans_df, window_len)

Next, you will create features that for each credit card aggregate data from multiple time steps.

Yoy will start by computing the distance between consecutive transactions, lets call it `loc_delta`.
Here you will use the [Haversine distance](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.haversine_distances.html?highlight=haversine#sklearn.metrics.pairwise.haversine_distances) to quantify the distance between two longitude and latitude coordinates.

Next lets compute windowed aggregates. Here you will use 4-hour windows, but feel free to experiment with different window lengths by setting `window_len` below to a value of your choice.

In [11]:
project = hopsworks.login()
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/327831
Connected. Call `.close()` to terminate connection gracefully.


To create a feature group you need to give it a name and specify a primary key. It is also good to provide a description of the contents of the feature group and a version number, if it is not defined it will automatically be incremented to `1`.

In [12]:
trans_fg = fs.get_feature_group(name="cc_trans_fraud", version=2)
trans_fg.insert(trans_df, write_options={"wait_for_job" : False})

Uploading Dataframe: 100.00% |██████████| Rows 60086/60086 | Elapsed Time: 00:10 | Remaining Time: 00:00


Launching job: cc_trans_fraud_2_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/327831/jobs/named/cc_trans_fraud_2_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7fd0678666e0>, None)

In [13]:
window_aggs_fg = fs.get_feature_group(name=f"cc_trans_fraud_{window_len}h", version=2)
window_aggs_fg.insert(window_aggs_df, write_options={"wait_for_job" : False})

Uploading Dataframe: 100.00% |██████████| Rows 60086/60086 | Elapsed Time: 00:07 | Remaining Time: 00:00


Launching job: cc_trans_fraud_4h_2_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/327831/jobs/named/cc_trans_fraud_4h_2_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7fd0677b17b0>, None)

In [14]:

labels_fg = fs.get_feature_group(name="transactions_fraud_label", version=2)
labels_fg.insert(fraud_labels)

Uploading Dataframe: 100.00% |██████████| Rows 60086/60086 | Elapsed Time: 00:07 | Remaining Time: 00:00


Launching job: transactions_fraud_label_2_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/327831/jobs/named/transactions_fraud_label_2_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7fd064dd5a50>, None)